In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
confirmed_data = pd.read_csv("time_series_covid_19_confirmed.csv", sep=',',header=0)
deaths_data = pd.read_csv("time_series_covid_19_deaths.csv", sep=',',header=0)
recovered_data = pd.read_csv("time_series_covid_19_recovered.csv", sep=',',header=0)

In [3]:
confirmed_data.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,47,48,50,50,50,53,59,70,75,82
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,360,420,461,502,511,581,639,639,701,773
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,117,130,138,150,150,160,178,178,200,212
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,50,83,93,99,117,129,149,149,197,238


In [4]:
confirmed_data = confirmed_data.drop(['Province/State', 'Lat', 'Long'], axis=1)
deaths_data = deaths_data.drop(['Province/State', 'Lat', 'Long'], axis=1)
recovered_data = recovered_data.drop(['Province/State', 'Lat', 'Long'], axis=1)

In [5]:
confirmed_data.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,Thailand,2,3,5,7,8,8,14,14,14,...,47,48,50,50,50,53,59,70,75,82
1,Japan,2,1,2,2,4,4,7,7,11,...,360,420,461,502,511,581,639,639,701,773
2,Singapore,0,1,3,3,4,5,7,7,10,...,117,130,138,150,150,160,178,178,200,212
3,Nepal,0,0,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,Malaysia,0,0,0,3,4,4,4,7,8,...,50,83,93,99,117,129,149,149,197,238


In [6]:
confirmed_data_countrywise = confirmed_data.groupby(['Country/Region']).sum()
deaths_data_countrywise = deaths_data.groupby(['Country/Region']).sum()
recovered_data_countrywise = recovered_data.groupby(['Country/Region']).sum()

In [7]:
confirmed_data_countrywise.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,1,1,1,4,4,5,7,7,7,11
Albania,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,10,12,23,33,38
Algeria,0,0,0,0,0,0,0,0,0,0,...,12,17,17,19,20,20,20,24,26,37
Andorra,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [8]:
confirmed_data_countrywise = confirmed_data_countrywise.reset_index()
deaths_data_countrywise = deaths_data_countrywise.reset_index()
recovered_data_countrywise = recovered_data_countrywise.reset_index()

In [14]:
confirmed_data_countrywise.loc[confirmed_data_countrywise['Country/Region']=='US']

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
135,US,1,1,2,2,5,5,5,5,5,...,217,262,402,518,583,959,1281,1663,2179,2727


In [15]:
cols = 5
rows = 29
plt.rcParams['figure.figsize'] = [cols*5, rows*5]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['axes.titleweight'] = 'bold'
fig, axs = plt.subplots(rows,cols)
i = 0
j = 0
countries = confirmed_data_countrywise.iloc[:,0].to_numpy()
for col in countries:
    arr = confirmed_data_countrywise.loc[confirmed_data_countrywise['Country/Region']==col].to_numpy()
    y = np.arange(np.size(arr,1)-1)
    axs[i,j].scatter(y, arr[0][1:], s=5)
    axs[i,j].set_title(col)
    i = (i+1)%rows
    if(i%rows==0):
        j = (j+1)%cols